# Labelme转COCO-批量

同济子豪兄 呕心沥血写成

兼容目标检测、图像分割、关键点检测三种标注

2023-3-9

2023-3-10

2023-4-15

2023-4-16

## 导入工具包

In [1]:
import os
import json
import numpy as np

## 删除系统自动生成的多余文件

### 查看待删除的多余文件

In [2]:
!find . -iname '__MACOSX'

In [3]:
!find . -iname '.DS_Store'

In [4]:
!find . -iname '.ipynb_checkpoints'

./【C】Labelme转COCO-单个文件/output_coco/.ipynb_checkpoints
./【C】Labelme转COCO-单个文件/.ipynb_checkpoints
./.ipynb_checkpoints


### 删除多余文件

In [5]:
!for i in `find . -iname '__MACOSX'`; do rm -rf $i;done

In [6]:
!for i in `find . -iname '.DS_Store'`; do rm -rf $i;done

In [7]:
!for i in `find . -iname '.ipynb_checkpoints'`; do rm -rf $i;done

### 验证多余文件已删除

In [8]:
!find . -iname '__MACOSX'

In [9]:
!find . -iname '.DS_Store'

In [10]:
!find . -iname '.ipynb_checkpoints'

## 指定数据集信息

In [11]:
Dataset_root = 'SJB_25_Dataset'

In [12]:
class_list = {
    'supercategory': 'sjb_rect',
    'id': 1,
    'name': 'sjb_rect',
    'keypoints': ['angle_30', 'angle_60', 'angle_90'], # 大小写敏感
    'skeleton':[[0,1], [0,2], [1,2]]
}

## 函数-处理单个labelme标注json文件

In [13]:
def process_single_json(labelme, image_id=1):
    '''
    输入labelme的json数据，输出coco格式的每个框的关键点标注信息
    '''
    
    global ANN_ID
    
    coco_annotations = []
    
    for each_ann in labelme['shapes']: # 遍历该json文件中的所有标注

        if each_ann['shape_type'] == 'rectangle': # 筛选出个体框

            # 个体框元数据
            bbox_dict = {}
            bbox_dict['category_id'] = 1
            bbox_dict['segmentation'] = []
            
            bbox_dict['iscrowd'] = 0
            bbox_dict['segmentation'] = []
            bbox_dict['image_id'] = image_id
            bbox_dict['id'] = ANN_ID
            # print(ANN_ID)
            ANN_ID += 1

            # 获取个体框坐标
            bbox_left_top_x = min(int(each_ann['points'][0][0]), int(each_ann['points'][1][0]))
            bbox_left_top_y = min(int(each_ann['points'][0][1]), int(each_ann['points'][1][1]))
            bbox_right_bottom_x = max(int(each_ann['points'][0][0]), int(each_ann['points'][1][0]))
            bbox_right_bottom_y = max(int(each_ann['points'][0][1]), int(each_ann['points'][1][1]))
            bbox_w = bbox_right_bottom_x - bbox_left_top_x
            bbox_h = bbox_right_bottom_y - bbox_left_top_y
            bbox_dict['bbox'] = [bbox_left_top_x, bbox_left_top_y, bbox_w, bbox_h] # 左上角x、y、框的w、h
            bbox_dict['area'] = bbox_w * bbox_h
            
            # 筛选出分割多段线
            for each_ann in labelme['shapes']: # 遍历所有标注
                if each_ann['shape_type'] == 'polygon': # 筛选出分割多段线标注
                    # 第一个点的坐标
                    first_x = each_ann['points'][0][0]
                    first_y = each_ann['points'][0][1]
                    if (first_x>bbox_left_top_x) & (first_x<bbox_right_bottom_x) & (first_y<bbox_right_bottom_y) & (first_y>bbox_left_top_y): # 筛选出在该个体框中的关键点
                        bbox_dict['segmentation'] = list(map(lambda x: list(map(lambda y: round(y, 2), x)), each_ann['points'])) # 坐标保留两位小数
                        # bbox_dict['segmentation'] = each_ann['points']

            # 筛选出该个体框中的所有关键点
            bbox_keypoints_dict = {}
            for each_ann in labelme['shapes']: # 遍历所有标注
                
                if each_ann['shape_type'] == 'point': # 筛选出关键点标注
                    # 关键点横纵坐标
                    x = int(each_ann['points'][0][0])
                    y = int(each_ann['points'][0][1])
                    label = each_ann['label']
                    if (x>bbox_left_top_x) & (x<bbox_right_bottom_x) & (y<bbox_right_bottom_y) & (y>bbox_left_top_y): # 筛选出在该个体框中的关键点
                        bbox_keypoints_dict[label] = [x, y]
                        
            bbox_dict['num_keypoints'] = len(bbox_keypoints_dict)
            # print(bbox_keypoints_dict)

            # 把关键点按照类别顺序排好
            bbox_dict['keypoints'] = []
            for each_class in class_list['keypoints']:
                if each_class in bbox_keypoints_dict:
                    bbox_dict['keypoints'].append(bbox_keypoints_dict[each_class][0])
                    bbox_dict['keypoints'].append(bbox_keypoints_dict[each_class][1])
                    bbox_dict['keypoints'].append(2) # 2-可见不遮挡 1-遮挡 0-没有点
                else: # 不存在的点，一律为0
                    bbox_dict['keypoints'].append(0)
                    bbox_dict['keypoints'].append(0)
                    bbox_dict['keypoints'].append(0)
                    
            coco_annotations.append(bbox_dict)
            
    return coco_annotations

## 函数-转换当前目录下所有labelme格式的json文件

In [14]:
def process_folder():
    IMG_ID = 0
    ANN_ID = 0

    # 遍历所有 labelme 格式的 json 文件
    for labelme_json in os.listdir(): 

        if labelme_json.split('.')[-1] == 'json':

            with open(labelme_json, 'r', encoding='utf-8') as f:

                labelme = json.load(f)

                ## 提取图像元数据
                img_dict = {}
                img_dict['file_name'] = labelme['imagePath']
                img_dict['height'] = labelme['imageHeight']
                img_dict['width'] = labelme['imageWidth']
                img_dict['id'] = IMG_ID
                coco['images'].append(img_dict)

                ## 提取框和关键点信息
                coco_annotations = process_single_json(labelme, image_id=IMG_ID)
                coco['annotations'] += coco_annotations

                IMG_ID += 1

                print(labelme_json, '已处理完毕')

        else:
            pass

## 转换训练集的所有labelme标注文件

In [15]:
coco = {}

coco['categories'] = []
coco['categories'].append(class_list)

coco['images'] = []
coco['annotations'] = []

IMG_ID = 0
ANN_ID = 0

In [16]:
path = os.path.join(Dataset_root, 'labelme_jsons', 'train_labelme_jsons')
os.chdir(path)

In [17]:
process_folder()

DSC_0283.json 已处理完毕
DSC_0293.json 已处理完毕
DSC_0301.json 已处理完毕
DSC_0284.json 已处理完毕
MVIMG_20230331_080908.json 已处理完毕
DSC_0259.json 已处理完毕
DSC_0289.json 已处理完毕
MVIMG_20230331_080914.json 已处理完毕
DSC_0209.json 已处理完毕
MVIMG_20230331_080912.json 已处理完毕
DSC_0285.json 已处理完毕
DSC_0278.json 已处理完毕
DSC_0281.json 已处理完毕
MVIMG_20230331_080915.json 已处理完毕
DSC_0240.json 已处理完毕
MVIMG_20230331_080920.json 已处理完毕
DSC_0282.json 已处理完毕
DSC_0236.json 已处理完毕
DSC_0269.json 已处理完毕
DSC_0219.json 已处理完毕


In [18]:
# 保存coco标注文件
coco_path = '../../train_coco.json'
with open(coco_path, 'w') as f:
    json.dump(coco, f, indent=2)
    
os.chdir('../../')

## 验证训练集的MS COCO格式标注

In [19]:
os.listdir()

['images', 'labelme_jsons', 'train_coco.json']

In [20]:
from pycocotools.coco import COCO

my_coco = COCO('train_coco.json')

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


## 转换测试集的所有labelme标注文件

In [21]:
coco = {}

coco['categories'] = []
coco['categories'].append(class_list)

coco['images'] = []
coco['annotations'] = []

IMG_ID = 0
ANN_ID = 0

In [22]:
path = os.path.join('labelme_jsons', 'val_labelme_jsons')
os.chdir(path)

In [23]:
process_folder()

DSC_0245.json 已处理完毕
DSC_0260.json 已处理完毕
DSC_0280.json 已处理完毕
DSC_0274.json 已处理完毕
DSC_0265.json 已处理完毕


In [24]:
# 保存coco标注文件
coco_path = '../../val_coco.json'
with open(coco_path, 'w') as f:
    json.dump(coco, f, indent=2)
    
os.chdir('../../')

## 验证测试集的MS COCO格式标注

In [25]:
from pycocotools.coco import COCO

my_coco = COCO('val_coco.json')

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


## 删除labelme标注目录

In [26]:
!rm -rf labelme_jsons

In [27]:
os.chdir('../')

## 删除系统自动生成的多余文件

### 查看待删除的多余文件

In [28]:
!find . -iname '__MACOSX'

In [29]:
!find . -iname '.DS_Store'

In [30]:
!find . -iname '.ipynb_checkpoints'

### 删除多余文件

In [31]:
!for i in `find . -iname '__MACOSX'`; do rm -rf $i;done

In [32]:
!for i in `find . -iname '.DS_Store'`; do rm -rf $i;done

In [33]:
!for i in `find . -iname '.ipynb_checkpoints'`; do rm -rf $i;done

### 验证多余文件已删除

In [34]:
!find . -iname '__MACOSX'

In [35]:
!find . -iname '.DS_Store'

In [36]:
!find . -iname '.ipynb_checkpoints'

## 得到完整的MS COCO格式的数据集

In [38]:
import seedir as sd
sd.seedir('SJB_25_Dataset/', style='emoji', depthlimit=2)

📁 SJB_25_Dataset/
├─📁 images/
│ ├─📄 DSC_0219.jpg
│ ├─📄 DSC_0209.jpg
│ ├─📄 DSC_0236.jpg
│ ├─📄 DSC_0293.jpg
│ ├─📄 DSC_0245.jpg
│ ├─📄 DSC_0278.jpg
│ ├─📄 MVIMG_20230331_080908.jpg
│ ├─📄 MVIMG_20230331_080920.jpg
│ ├─📄 DSC_0285.jpg
│ ├─📄 DSC_0284.jpg
│ ├─📄 DSC_0280.jpg
│ ├─📄 DSC_0281.jpg
│ ├─📄 DSC_0240.jpg
│ ├─📄 DSC_0283.jpg
│ ├─📄 DSC_0269.jpg
│ ├─📄 DSC_0282.jpg
│ ├─📄 DSC_0259.jpg
│ ├─📄 MVIMG_20230331_080914.jpg
│ ├─📄 MVIMG_20230331_080915.jpg
│ ├─📄 DSC_0289.jpg
│ ├─📄 DSC_0260.jpg
│ ├─📄 DSC_0274.jpg
│ ├─📄 DSC_0301.jpg
│ ├─📄 DSC_0265.jpg
│ └─📄 MVIMG_20230331_080912.jpg
├─📄 train_coco.json
└─📄 val_coco.json
